## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv", index_col=[0])
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Presidente Epitacio,BR,-21.7633,-52.1156,66.96,48,0,6.13,clear sky
1,Silvan,TR,38.1419,41.0081,71.89,18,0,0.20,clear sky
2,Hermanus,ZA,-34.4187,19.2345,48.79,85,31,6.26,scattered clouds
3,Eureka,US,40.8021,-124.1637,57.61,87,45,3.11,scattered clouds
4,Rikitea,PF,-23.1203,-134.9692,74.88,70,74,2.91,light rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip?"))
max_temp=float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_locations_df=city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & \
                                      (city_data_df["Max Temp"]>=min_temp)]
preferred_locations_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,Ixtapa,MX,20.7000,-105.2000,75.22,100,100,4.61,overcast clouds
24,Portland,US,45.5234,-122.6762,89.55,53,0,5.75,clear sky
28,Atuona,PF,-9.8000,-139.0333,77.56,74,12,17.67,few clouds
35,Hithadhoo,MV,-0.6000,73.0833,76.77,87,100,9.78,moderate rain
38,Dingle,PH,10.9995,122.6711,87.08,71,88,12.86,overcast clouds
47,Lamesa,US,32.7376,-101.9510,86.52,32,15,18.63,few clouds
50,Faanui,PF,-16.4833,-151.7500,78.58,72,22,7.54,few clouds
53,Ternate,ID,0.8000,127.4000,81.50,79,100,4.54,overcast clouds
58,Kailaras,IN,26.3167,77.6167,88.59,62,86,8.25,overcast clouds
59,Katsuura,JP,35.1333,140.3000,83.23,80,40,15.23,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_locations_df.count()
bool_series=pd.isnull(preferred_locations_df["Max Temp"])
preferred_locations_df[bool_series]

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description


In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_locations_df.count()

City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [12]:
clean_df=preferred_locations_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ixtapa,MX,75.22,overcast clouds,20.7000,-105.2000,
24,Portland,US,89.55,clear sky,45.5234,-122.6762,
28,Atuona,PF,77.56,few clouds,-9.8000,-139.0333,
35,Hithadhoo,MV,76.77,moderate rain,-0.6000,73.0833,
38,Dingle,PH,87.08,overcast clouds,10.9995,122.6711,
47,Lamesa,US,86.52,few clouds,32.7376,-101.9510,
50,Faanui,PF,78.58,few clouds,-16.4833,-151.7500,
53,Ternate,ID,81.50,overcast clouds,0.8000,127.4000,
58,Kailaras,IN,88.59,overcast clouds,26.3167,77.6167,
59,Katsuura,JP,83.23,scattered clouds,35.1333,140.3000,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,"type": "lodging","key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]=f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotels=requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ixtapa,MX,75.22,overcast clouds,20.7000,-105.2000,hotel la terminal
24,Portland,US,89.55,clear sky,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
28,Atuona,PF,77.56,few clouds,-9.8000,-139.0333,Villa Enata
35,Hithadhoo,MV,76.77,moderate rain,-0.6000,73.0833,Scoop Guest House
38,Dingle,PH,87.08,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
...,...,...,...,...,...,...,...
668,Price,US,79.86,broken clouds,39.5994,-110.8107,National 9 - Price River Inn
669,Rincon,US,82.85,clear sky,32.2960,-81.2354,Days Inn Rincon
671,Todos Santos,MX,78.04,scattered clouds,23.4500,-110.2167,Hotel California Todos Santos
673,Kencong,ID,83.10,few clouds,-8.2833,113.3667,GUBUK SUGIANTO


In [18]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
current_description = hotel_df["Current Description"]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))